In [1]:
!pip install faker 
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Hetvi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
fake = Faker('en_CA')  # Use Canadian locale

In [3]:
def generate_customers(n):
    customers = []
    for i in range(1, n + 1):
        customers.append({
            'customer_id': i,
            'first_name': fake.first_name(),
            'last_name': fake.last_name(),
            'email': fake.unique.email(),
            'phone_number': fake.phone_number(),
            'address': fake.street_address(),
            'city': fake.city(),
            'province': fake.province(),
            'postal_code': fake.postalcode(),
            'signup_date': fake.date_between(start_date='-5y', end_date='today')
        })
    return pd.DataFrame(customers)


In [4]:
def generate_services():
    services = [
        {'service_id': 1, 'service_name': 'Mobile Basic', 'service_type': 'Mobile', 'monthly_cost': 30.00},
        {'service_id': 2, 'service_name': 'Mobile Plus', 'service_type': 'Mobile', 'monthly_cost': 50.00},
        {'service_id': 3, 'service_name': 'Internet Basic', 'service_type': 'Internet', 'monthly_cost': 40.00},
        {'service_id': 4, 'service_name': 'Internet Fiber', 'service_type': 'Internet', 'monthly_cost': 70.00},
        {'service_id': 5, 'service_name': 'TV Basic', 'service_type': 'TV', 'monthly_cost': 25.00},
        {'service_id': 6, 'service_name': 'TV Premium', 'service_type': 'TV', 'monthly_cost': 55.00},
    ]
    return pd.DataFrame(services)


In [5]:
def generate_subscriptions(customers_df, services_df):
    subscriptions = []
    subscription_id = 1
    for index, customer in customers_df.iterrows():
        num_subscriptions = random.randint(1, 3)  # Each customer can have 1 to 3 services
        service_choices = services_df['service_id'].sample(num_subscriptions).tolist()
        for service_id in service_choices:
            start_date = customer['signup_date']
            status = random.choice(['Active', 'Inactive'])
            end_date = None if status == 'Active' else fake.date_between(start_date=start_date, end_date='today')
            
            # Get service_type from services_df
            service_type = services_df.loc[services_df['service_id'] == service_id, 'service_type'].values[0]
            
            subscriptions.append({
                'subscription_id': subscription_id,
                'customer_id': customer['customer_id'],
                'service_id': service_id,
                'service_type': service_type,  # Include service_type
                'start_date': start_date,
                'end_date': end_date,
                'status': status
            })
            subscription_id += 1
    return pd.DataFrame(subscriptions)


In [6]:
def generate_usage(subscriptions_df):
    usage_records = []
    usage_id = 1
    for index, subscription in subscriptions_df.iterrows():
        if subscription['status'] == 'Active':
            num_records = random.randint(5, 15)  # Number of usage records per subscription
            for _ in range(num_records):
                usage_date = fake.date_between(start_date=subscription['start_date'], end_date='today')
                data_used = random.uniform(0.1, 5.0) if subscription['service_type'] == 'Mobile' else random.uniform(1.0, 50.0)
                minutes_used = random.randint(1, 500)
                usage_records.append({
                    'usage_id': usage_id,
                    'subscription_id': subscription['subscription_id'],
                    'usage_date': usage_date,
                    'data_used_gb': round(data_used, 2),
                    'minutes_used': minutes_used
                })
                usage_id += 1
    return pd.DataFrame(usage_records)


In [7]:
def generate_billing(subscriptions_df):
    billing_records = []
    billing_id = 1
    for index, subscription in subscriptions_df.iterrows():
        if subscription['status'] == 'Active':
            num_bills = random.randint(1, 12)  # Number of billing cycles
            bill_date = subscription['start_date']
            for _ in range(num_bills):
                amount_due = services_df.loc[services_df['service_id'] == subscription['service_id'], 'monthly_cost'].values[0]
                amount_paid = amount_due if random.random() > 0.1 else 0  # 10% chance of non-payment
                payment_date = bill_date + timedelta(days=random.randint(0, 10)) if amount_paid > 0 else None
                payment_method = random.choice(['Credit Card', 'Debit', 'Bank Transfer', 'Cash'])
                billing_records.append({
                    'billing_id': billing_id,
                    'subscription_id': subscription['subscription_id'],
                    'billing_date': bill_date,
                    'amount_due': amount_due,
                    'amount_paid': amount_paid,
                    'payment_date': payment_date,
                    'payment_method': payment_method if amount_paid > 0 else None
                })
                billing_id += 1
                bill_date += timedelta(days=30)
    return pd.DataFrame(billing_records)


In [8]:
def generate_support_tickets(customers_df):
    tickets = []
    ticket_id = 1
    for index, customer in customers_df.sample(frac=0.1).iterrows():  # 10% of customers
        num_tickets = random.randint(1, 5)
        for _ in range(num_tickets):
            issue_date = fake.date_between(start_date=customer['signup_date'], end_date='today')
            issue_type = random.choice(['Technical', 'Billing', 'General Inquiry', 'Cancellation'])
            resolution_date = issue_date + timedelta(days=random.randint(1, 7))
            status = random.choice(['Resolved', 'Open', 'Pending'])
            tickets.append({
                'ticket_id': ticket_id,
                'customer_id': customer['customer_id'],
                'issue_date': issue_date,
                'issue_type': issue_type,
                'resolution_date': resolution_date if status == 'Resolved' else None,
                'status': status
            })
            ticket_id += 1
    return pd.DataFrame(tickets)


In [9]:
# Generate data
print("Generating customers...")
customers_df = generate_customers(100000)

print("Generating services...")
services_df = generate_services()

print("Generating subscriptions...")
subscriptions_df = generate_subscriptions(customers_df, services_df)

print("Generating usage...")
usage_df = generate_usage(subscriptions_df)

print("Generating billing...")
billing_df = generate_billing(subscriptions_df)

print("Generating support tickets...")
support_tickets_df = generate_support_tickets(customers_df)

# Save to CSV
print("Saving to CSV...")
customers_df.to_csv('customers.csv', index=False)
services_df.to_csv('services.csv', index=False)
subscriptions_df.to_csv('subscriptions.csv', index=False)
usage_df.to_csv('usage.csv', index=False)
billing_df.to_csv('billing.csv', index=False)
support_tickets_df.to_csv('support_tickets.csv', index=False)


Generating customers...
Generating services...
Generating subscriptions...
Generating usage...
Generating billing...
Generating support tickets...
Saving to CSV...


In [22]:
billing_df.head(3)

,billing_id,subscription_id,billing_date,amount_due,amount_paid,payment_date,payment_method
0,1,1,2022-03-09,40.0,40.0,2022-03-19,Cash
1,2,1,2022-04-08,40.0,40.0,2022-04-15,Credit Card
2,3,1,2022-05-08,40.0,40.0,2022-05-08,Bank Transfer


In [15]:
customers_df.head(3)

,customer_id,first_name,last_name,email,phone_number,address,city,province,postal_code,signup_date
0,1,Thomas,Whitaker,morgannancy@example.org,629-234-4467,27269 Alexander Lodge Suite 107,West Carlos,New Brunswick,P8E 1P9,2022-03-09
1,2,Matthew,Hughes,rebeccafoley@example.org,1 (959) 948-1473,8892 Schultz Drive,Pittsbury,Quebec,X7C3T4,2020-04-26
2,3,Erin,Stewart,grantmegan@example.com,915.260.8094,37746 Mark Prairie,New Michelle,Prince Edward Island,E5M 3R4,2022-12-12


In [28]:
services_df.head()

,service_id,service_name,service_type,monthly_cost
0,1,Mobile Basic,Mobile,30.0
1,2,Mobile Plus,Mobile,50.0
2,3,Internet Basic,Internet,40.0
3,4,Internet Fiber,Internet,70.0
4,5,TV Basic,TV,25.0


In [ ]:
subscriptions_df.head(3)

,subscription_id,customer_id,service_id,service_type,start_date,end_date,status
0,1,1,3,Internet,2022-03-09,None,Active
1,2,1,5,TV,2022-03-09,None,Active
2,3,2,2,Mobile,2020-04-26,2020-07-19,Inactive


In [ ]:
support_tickets_df.head(3)

,ticket_id,customer_id,issue_date,issue_type,resolution_date,status
0,1,74027,2024-04-24,General Inquiry,2024-05-01,Resolved
1,2,74027,2023-11-30,Cancellation,2023-12-07,Resolved
2,3,74027,2023-09-19,Technical,2023-09-23,Resolved


In [19]:
usage_df.head(3)

,usage_id,subscription_id,usage_date,data_used_gb,minutes_used
0,1,1,2022-12-26,22.75,176
1,2,1,2023-10-12,43.50,236
2,3,1,2023-09-29,28.48,66
